In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rn
from scipy.spatial import distance
from scipy import stats
from scipy.stats import expon
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import urlopen
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
year_before = 2015
year_after = 2018

In [ ]:
df_traits = pd.read_csv('Data_sets/Aggregated_network_attributes_period_'+str(year_before)+'.csv', index_col=False)
df_traits['Household Size'] = df_traits['Household Size'].div(5)

In [ ]:
df_hesitancy = pd.read_csv('Data_sets/Aggregated_network_hesitancy_level_period_'+str(year_before)+'-'+str(year_after)+'.csv', index_col=False)

In [ ]:
df_facebook_friendship = pd.read_csv('Data_sets/Aggregated_facebook_network_period_'+str(year_before)+'-'+str(year_after)+'.csv', index_col=False)

In [ ]:
df_proximity = pd.read_csv('Data_sets/Aggregated_spatial_network_period_'+str(year_before)+'-'+str(year_after)+'.csv', index_col=False)

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# CREATE NETWORKS

In [ ]:
spatial_network = nx.from_pandas_edgelist(df_proximity, source = 'source', target = 'target')
spatial_network.remove_edges_from(list(nx.selfloop_edges(spatial_network)))

In [ ]:
facebook_network = nx.from_pandas_edgelist(df_facebook_friendship, source = 'source', target = 'target')
facebook_network.remove_edges_from(list(nx.selfloop_edges(facebook_network)))    

In [ ]:
spatial_network.add_nodes_from([each_node for each_node in facebook_network.nodes if each_node not in spatial_network.nodes]) 
facebook_network.add_nodes_from([each_node for each_node in spatial_network.nodes if each_node not in facebook_network.nodes]) 

In [ ]:
nodes = list(df_traits['FIPS'])
income_dict = dict(zip(nodes, list(df_traits['High Income'])))
household_dict = dict(zip(nodes, list(df_traits['Household Size'])))

nodes = list(df_hesitancy['FIPS'])
hesitancy_before_dict = dict(zip(nodes, list(df_hesitancy['Hesitancy level before'])))
hesitancy_after_dict = dict(zip(nodes, list(df_hesitancy['Hesitancy level after'])))
opinion_before_dict = dict(zip(nodes, list(df_hesitancy['Opinion before'])))
opinion_after_dict = dict(zip(nodes, list(df_hesitancy['Opinion after'])))

In [ ]:
# ASIGN ATTRIBUTES TO COUNTIES SPATIAL NETWORK
dict_list = [income_dict,household_dict, hesitancy_before_dict,\
             hesitancy_after_dict,opinion_before_dict,opinion_after_dict]

names = ["High Income","Household Size","Hesitancy Level before",\
         "Hesitancy Level after","Opinion Level before","Opinion Level after"]

for each_attribute in range(0,6): 
    
    # ASIGN ATTRIBUTES TO COUNTIES SPATIAL NETWORK
    nx.set_node_attributes(spatial_network,dict_list[each_attribute],names[each_attribute])
    # ASIGN ATTRIBUTES TO COUNTIES FACEBOOK NETWORK
    nx.set_node_attributes(facebook_network,dict_list[each_attribute],names[each_attribute])

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

In [ ]:
df_parameters = pd.DataFrame(columns = ['Years','Data','Parameter value','Realization'])

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# BOOTSTRAP METHOD TO MEASURE EMPIRICAL SOCIAL SELECTION

## RANDOM

In [ ]:
for realization in range(1,1000):
    
    bootstrap_random_network = facebook_network.copy()
    
    income_shuffle = rn.sample(list(df_traits['High Income']),len(list(df_traits['High Income'])))
    household_shuffle = rn.sample(list(df_traits['Household Size']),len(list(df_traits['Household Size'])))
    
    income_dict = dict(zip(nodes, income_shuffle))
    household_dict = dict(zip(nodes, household_shuffle))
 
    nx.set_node_attributes(bootstrap_random_network, income_dict, 'High Income')
    nx.set_node_attributes(bootstrap_random_network, household_dict, 'Household Size')

    
    xi, xj = [], []
    for i,j in bootstrap_random_network.edges:
        xi.append([bootstrap_random_network.nodes[i]['High Income'], bootstrap_random_network.nodes[i]['Household Size']])
        xj.append([bootstrap_random_network.nodes[j]['High Income'], bootstrap_random_network.nodes[j]['Household Size']])   
    
    income = [bootstrap_random_network.nodes[i]['High Income'] for i in bootstrap_random_network.nodes]    
    house  = [bootstrap_random_network.nodes[i]['Household Size'] for i in bootstrap_random_network.nodes] 
    corr   = np.cov(income,house)
    vi     = np.linalg.inv(corr)        
    
    distances = [ distance.mahalanobis( xi[i], xj[i], vi ) for i in range(len(xj)) ]
    
    random_distance = sum(distances)/len(distances)

## REAL

In [ ]:
porcentaje = int(0.3 * facebook_network.number_of_nodes())
nodes = [each_node for each_node in facebook_network.nodes()]

for realization in range(1,1000):
    
    graph_bootstrap = facebook_network.copy()
    sample = rn.sample(nodes,porcentaje)
    graph_bootstrap.remove_nodes_from(sample)
    
    xi, xj = [], []
    for i,j in graph_bootstrap.edges:
        xi.append([graph_bootstrap.nodes[i]['High Income'], graph_bootstrap.nodes[i]['Household Size']])
        xj.append([graph_bootstrap.nodes[j]['High Income'], graph_bootstrap.nodes[j]['Household Size']])   
    
    income = [graph_bootstrap.nodes[i]['High Income'] for i in graph_bootstrap.nodes]    
    house  = [graph_bootstrap.nodes[i]['Household Size'] for i in graph_bootstrap.nodes] 
    corr   = np.cov(income,house)
    vi     = np.linalg.inv(corr)        
    
    distances = [ distance.mahalanobis( xi[i], xj[i], vi ) for i in range(len(xj)) ]
    
    mahalanobi_distance = sum(distances)/len(distances)
    random_distance = 1.7
    social_selection = round(1 - mahalanobi_distance / random_distance, 2)
    
    
    df_parameters = df_parameters.append({'Years': '2015-2018', 'Data': 'Empirical Social Selection', 'Parameter value' : social_selection, 'Realization' : realization}, ignore_index = True)

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# BOOTSTRAP METHOD TO MEASURE EMPIRICAL SOCIAL INFLUENCE

In [ ]:
porcentaje = int(0.3 * facebook_network.number_of_nodes())
nodes = [each_node for each_node in facebook_network.nodes()]


for realization in range(1,1000):
    
    graph_bootstrap = facebook_network.copy()
    sample = rn.sample(nodes,porcentaje)
    graph_bootstrap.remove_nodes_from(sample)

    hesitancy_distribution = [graph_bootstrap.nodes[n]['Hesitancy Level before'] for n in graph_bootstrap.nodes()]
    
    increasing_nodes = [each_node for each_node in graph_bootstrap.nodes if (graph_bootstrap.nodes[each_node]['Hesitancy Level before'] < graph_bootstrap.nodes[each_node]['Hesitancy Level after']) ]
    
    number_on_increasing_nodes = len(increasing_nodes)
    
    fraction = [((sum([(graph_bootstrap.nodes[each_neighbor]['Hesitancy Level before']) for each_neighbor in graph_bootstrap.neighbors(each_node)]) + graph_bootstrap.nodes[each_node]['Hesitancy Level before']) / (graph_bootstrap.degree(each_node) + 1)) for each_node in increasing_nodes]
    
    multiplier = np.percentile(np.array(hesitancy_distribution),95)

    influence_value = sum(fraction) / len(fraction) / multiplier    

    df_parameters  = df_parameters.append({'Years': '2015-2018', 'Data': 'Empirical Social Influence', 'Parameter value' : influence_value, 'Realization' : realization}, ignore_index = True)

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# BOOTSTRAP METHOD TO MEASURE EMPIRICAL SPATIAL CLUSTERING 2018

In [ ]:
porcentaje = int(0.3 * spatial_network.number_of_nodes())
nodes = [each_node for each_node in spatial_network.nodes()]

for rea in range(1,1000):
    
    graph_bootstrap = spatial_network.copy()
    sample = rn.sample(nodes,porcentaje)
    graph_bootstrap.remove_nodes_from(sample)
    
    SC = nx.attribute_assortativity_coefficient(graph_bootstrap,'Opinion Level after')

    df_parameters  = df_parameters.append({'Years': '2015-2018', 'Data': 'Empirical Spatial Clustering', 'Parameter value' : SC, 'Realization' : realization}, ignore_index = True)

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# SIMULATED SPATIAL CLUSTERING 2015-2018

In [ ]:
influence = df_parameters[df_parameters['Data'] == 'Empirical Spatial Clustering']['Parameter value'].mean() * multiplier
vulnerables_after = len([each_node for each_node in facebook_network.nodes if facebook_network.nodes[each_node]['Opinion Level after'] == 'Vulnerable'])
jump = 0.01
cutoff = 0.05

for rea in range(1,1000,1):

    graph = facebook_network.copy()
    graph_regular = spatial_network.copy()
    
    protected = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Protected']
    vulnerable = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Vulnerable']
    
    noisy_nodes = [] 
    nodes_to_change = 2
    noisy_interval = 4
    time = 0
    
    while True:
        
        index = 0

        protected = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Protected']
        vulnerables = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Vulnerable']
        
        fraction = [ [each_node, ((sum([(graph.nodes[each_neighbor]['Hesitancy Level before']) for each_neighbor in graph.neighbors(each_node)])+graph.nodes[each_node]['Hesitancy Level before']) / (graph.degree(each_node)+1))] for each_node in graph.nodes ]
        result = [ [x,y] for x, y in fraction if (influence < y) ] 
        result.sort(key = lambda x: x[1], reverse = True)
        
        vulnerables_increasing = [each_node for each_node, refusal in result if graph.nodes[each_node]['Opinion Level before'] == 'Vulnerable']
        for each_node in vulnerables_increasing: graph.nodes[each_node]['Hesitancy Level before'] = graph.nodes[each_node]['Hesitancy Level before'] + jump

        protected_increasing = [each_node for each_node, refusal in result if (graph.nodes[each_node]['Opinion Level before'] == 'Protected' and graph.nodes[each_node]['Hesitancy Level before'] + jump < cutoff)]
        for each_node in protected_increasing: graph.nodes[each_node]['Hesitancy Level before'] = graph.nodes[each_node]['Hesitancy Level before'] + jump

        protected_changing = [each_node for each_node, refusal in result if \
                                (graph.nodes[each_node]['Opinion Level before'] == 'Protected' and \
                                 graph.nodes[each_node]['Hesitancy Level before'] + jump >= cutoff)]
        
        for each_node in protected_changing:
            if (len(vulnerables) < (vulnerables_after - nodes_to_change)):
                graph.nodes[each_node]['Hesitancy Level before'] = graph.nodes[each_node]['Hesitancy Level before'] + jump
                graph.nodes[each_node]['Opinion Level before'] = 'Vulnerable'
                vulnerables.append(each_node)
            else:
                noisy_nodes.append(each_node)
                index = len(noisy_nodes)

        if index >= nodes_to_change:
            selected_noisy = rn.sample(noisy_nodes[:noisy_interval],nodes_to_change)            
            for each_node in selected_noisy:
                graph.nodes[each_node]['Hesitancy Level before'] = graph.nodes[each_node]['Hesitancy Level before'] + jump
                graph.nodes[each_node]['Opinion Level before'] = 'Vulnerable'
                vulnerables.append(each_node)

        protected = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Protected']
        vulnerable = [each_node for each_node in graph.nodes if graph.nodes[each_node]['Opinion Level before'] == 'Vulnerable']   

        time = time + 1 
        if time == 1000 : break 
        if len(vulnerables) == vulnerables_after : break 
            
    for each_node in graph_regular.nodes: graph_regular.nodes[each_node]['Opinion Level before'] = graph.nodes[each_node]['Opinion Level before']   
    spatial_clustering = nx.attribute_assortativity_coefficient(graph_regular,'Opinion Level before')

    df_parameters  = df_parameters.append({'Years': '2015-2018', 'Data': 'Simulated Spatial Clustering', 'Parameter value' : spatial_clustering, 'Realization' : realization}, ignore_index = True)    

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# PARAMETER

In [ ]:
sns.set_style("whitegrid")
colors = ['#BF96FA','#7B57AD','#7DFA9B','#DFB9AB']
ax = sns.pointplot(data = df_parameters, x = "Data", y = "Parameter value", hue = "Data", linestyle = False, palette = colors, err_style ='bars', ci = 'sd', scale = 1.5)
plt.xticks([])
plt.yticks(fontsize=14)
plt.xlabel("Parameters", fontsize=16)
plt.ylabel("")
plt.legend(title =False,loc="upper left", fontsize=11)
ax.figure.savefig('5-Parameters.png',transparent=True,dpi=200)

######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

# MAPS OF COUNTIES STATUS

In [ ]:
year = 2015
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)   

df_hesitancy.FIPS = df_hesitancy.FIPS.astype(float).astype(int).astype(str).str.zfill(5)

fig = px.choropleth(df_hesitancy,               
                    geojson = counties,
                    locations = 'FIPS',
                    color = df_hesitancy['Opinion before'],
                    color_discrete_map = {'Vulnerable': "#DEB09E", 'Protected': "#418F78"},
                    scope = 'usa'
                   )
fig.update_layout(legend_title = "County Status")
fig.update_traces(marker_line_width = 0.1, 
                  marker_opacity = 0.85, 
                  marker_line_color = '#262626',
                  )
fig.write_image("4b-"+str(year)+".png", scale = 2)

In [ ]:
year = 2018
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)   

df_hesitancy.FIPS = df_hesitancy.FIPS.astype(float).astype(int).astype(str).str.zfill(5)

fig = px.choropleth(df_hesitancy,                 
                    geojson = counties,
                    locations = 'FIPS',
                    color = df_hesitancy['Opinion after'],
                    color_discrete_map = {'Vulnerable': "#DEB09E", 'Protected': "#418F78"},
                    scope = 'usa'
                   ) 
fig.update_layout(showlegend = False)
fig.update_traces(marker_line_width = 0.1,  
                  marker_opacity = 0.85, 
                  marker_line_color = '#262626',
                  )

fig.write_image("4b-"+str(year)+"-empirical.png", scale = 2)

In [ ]:
dict_simulated = {'FIPS' : [each_node for each_node in graph_regular.nodes],'County Status' : [graph_regular.nodes[each_node]['Opinion Level before'] for each_node in graph_regular.nodes]}
df_simulated = pd.DataFrame(dict_simulated )

In [ ]:
year = 2018
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)   

df_simulated.FIPS = df_simulated.FIPS.astype(float).astype(int).astype(str).str.zfill(5)

fig = px.choropleth(df_simulated,               
                    geojson = counties,
                    locations = 'FIPS',
                    color = df_simulated['County Status'],
                    color_discrete_map = {'Vulnerable': "#DEB09E", 'Protected': "#418F78"},
                    scope = 'usa'
                   )
fig.update_layout(showlegend = False)
fig.update_traces(marker_line_width = 0.1,
                  marker_opacity = 0.85, 
                  marker_line_color = '#262626',
                  )

fig.write_image("4b-"+str(year)+"-simulated.png", scale = 2)